In [1]:
from functions0944 import g_D_symbolic_coefficients_dict
from functions0944 import restore_matrices
from functions0944 import Augmented_Lagrangian
from functions0944 import update_Lagrangian_coefficients
from functions0944 import generate_M_d
from functions0944 import jac
import numpy as np
from scipy.integrate import quad
from functools import partial
import jax
import jax.numpy as jnp
from scipy.optimize import minimize


In [2]:
D = 2
d = 4
L = 2*d+1

In [3]:
expanded_result, coefficients_dict,polynomial,x = g_D_symbolic_coefficients_dict(D)
print("Expanded Result:")
print(expanded_result)
print("\nCoefficients Dictionary:")
for vars_tuple, coeff in coefficients_dict.items():
    print(f"Coefficient of {vars_tuple}: {coeff}")

orders_list = list(coefficients_dict.keys())
coefficients_list = list(coefficients_dict.values())
print(orders_list)
print(coefficients_list)

Expanded Result:
4.0*x1**4 + x1**3/8 + 3*x1**2*x2/8 - 4.0*x1**2 + 3*x1*x2**2/8 + 4.0*x2**4 + x2**3/8 - 4.0*x2**2 + 1.0

Coefficients Dictionary:
Coefficient of (4, 0): 4.0
Coefficient of (3, 0): 0.125
Coefficient of (2, 1): 0.375
Coefficient of (2, 0): -4.0
Coefficient of (1, 2): 0.375
Coefficient of (0, 4): 4.0
Coefficient of (0, 3): 0.125
Coefficient of (0, 2): -4.0
Coefficient of (0, 0): 1.0
[(4, 0), (3, 0), (2, 1), (2, 0), (1, 2), (0, 4), (0, 3), (0, 2), (0, 0)]
[4.0, 0.125, 0.375, -4.0, 0.375, 4.0, 0.125, -4.0, 1.0]


In [4]:
# This block is for initialization according to the article, µ, R_i^l,is initialized randomly using a distribution U ([−1, 1]) uniform over [−1, 1].

# def function of the moments in M_d
def f(x,n):
    return 1/2*x**n

In [5]:
# This block is for the matrix initialization and flatten as x_input

x_mu_D_L_list = [[] for _ in range(D)]
x_R_L_list = [[] for _ in range(D)]

# 9/28 update, we first generate a list of different moments of measure, then in the augumented lagrangian, we can generate the M_d matrix from our list of moments of measure
# This can fix the problem that when we try to differentiate the ϕn(µ), it actully differentiate the list of moments of measure not the M_d matrix 

# Define the mu_list
list_size = 2*d+3
my_list = np.array([quad(f,-1,1,args=(i))[0] for i in range(2*d+3)])

# generate the list of list of both matrix
for l in range(L):
    for i in range(D):
        x_mu_D_L_list[i].append(my_list)
        x_R_L_list[i].append(np.random.uniform(-1, 1, size=(d+1, d+1)))

x_matrix_list_new = x_mu_D_L_list+x_R_L_list
x_input = np.array([])
for matrix_index in range(len(x_matrix_list_new)):
    for l in range(L):
        flattened_array = x_matrix_list_new[matrix_index][l].flatten()
        x_input = np.concatenate((x_input, flattened_array))
print(len(x_input))

648


In [6]:
# Initialize the lagrangian coefficient as 0
Lagrangian_coefficient = 0
rho = 10

In [7]:
#implement the augumented lagrangian
aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                 coefficients_list=coefficients_list,
                                 Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)

#implement the augumented lagrangian
aug_jac_partial = partial(jac, d=d, D=D, L=L, orders_list=orders_list,
                                 coefficients_list=coefficients_list,
                                 Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)

#use jax autogradient to generate the gradient
aug_lagrangian_partial_gradient = jax.grad(aug_lagrangian_partial)



In [13]:
print(aug_jac_partial(x_input))
print(aug_lagrangian_partial_gradient(x_input))

[-5.101347478230794, 4.792695045471191, -6.68207049369812, -1.96482515335083, 10.272396087646484, 4.792695045471191, -14.197870254516602, 2.7178454399108887, 5.553484797477722, 2.154392719268799, -6.68207049369812, 2.7178454399108887, -14.771867752075195, -2.689671039581299, -4.125043869018555, -1.96482515335083, 5.553484797477722, -2.689671039581299, -0.692230761051178, -1.4131265878677368, 10.272396087646484, 2.154392719268799, -4.125043869018555, -1.4131265878677368, -5.251073360443115, -0.5465944131215412, -5.989693641662598, 3.352273464202881, -0.14923930168151855, 14.805023193359375, -5.989693641662598, -16.064708709716797, 1.9249696731567383, 8.305882453918457, 7.327206134796143, 3.352273464202881, 1.9249696731567383, -14.865480422973633, -1.0137455463409424, 4.888002395629883, -0.14923930168151855, 8.305882453918457, -1.0137455463409424, -12.011727333068848, -4.51298189163208, 14.805023193359375, 7.327206134796143, 4.888002395629883, -4.51298189163208, -12.58232307434082, -2.96

In [14]:
a = np.array([1,2,3])
b = np.array([1,2,3])
a*b

array([1, 4, 9])